## Imports

In [31]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os

## Data Read-in and clean

In [2]:
df = pd.read_csv('Manhattan_Brooklyn addresss with 15+ Hamptons owners - Sheet1.csv')

In [4]:
df = df.drop(columns='Unnamed: 0')

In [6]:
df['Full_Address'] = df['Address'] + ' New York, NY'

## Geocode

In [9]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [25]:
from shapely.geometry import Point
import geopandas as gpd

# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'Full_Address' column and store the results in the 'geocoded' column
df['geocoded'] = df['Full_Address'].apply(geocode)

# Split 'geocoded' into 'lat' and 'lon', handling None values
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x is None else x).apply(pd.Series)

# Convert 'lat' and 'lon' to floats
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

# Create the 'geometry' column with Point objects
df['geometry'] = df.apply(lambda row: Point(row['lon'], row['lat']) if pd.notnull(row['lat']) and pd.notnull(row['lon']) else None, axis=1)

# Drop rows where geometry is None
df = df.dropna(subset=['geometry'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set CRS (Coordinate Reference System) to WGS 84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)


/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:131: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,Address,Count,Full_Address,geocoded,lat,lon,geometry
0,1185 Park Avenue,33,"1185 Park Avenue New York, NY","(40.7846858, -73.9531111)",40.784686,-73.953111,POINT (-73.95311 40.78469)
1,500 East 77th Street,26,"500 East 77th Street New York, NY","(40.7695627, -73.95079989999999)",40.769563,-73.950800,POINT (-73.95080 40.76956)
2,425 East 58th Street,24,"425 East 58th Street New York, NY","(40.7588424, -73.9611658)",40.758842,-73.961166,POINT (-73.96117 40.75884)
3,115 Central Park West,23,"115 Central Park West New York, NY","(40.776224, -73.9766209)",40.776224,-73.976621,POINT (-73.97662 40.77622)
4,767 Fifth Avenue,22,"767 Fifth Avenue New York, NY","(40.7635906, -73.9724052)",40.763591,-73.972405,POINT (-73.97241 40.76359)
5,300 Central Park West,22,"300 Central Park West New York, NY","(40.7882884, -73.9674607)",40.788288,-73.967461,POINT (-73.96746 40.78829)
6,45 East 89th Street,20,"45 East 89th Street New York, NY","(40.78294530000001, -73.95662879999999)",40.782945,-73.956629,POINT (-73.95663 40.78295)
7,215 East 68th Street,20,"215 East 68th Street New York, NY","(40.7670517, -73.960883)",40.767052,-73.960883,POINT (-73.96088 40.76705)
8,200 East 66th Street,20,"200 East 66th Street New York, NY","(40.7652576, -73.96250789999999)",40.765258,-73.962508,POINT (-73.96251 40.76526)
9,95 Horatio Street,17,"95 Horatio Street New York, NY","(40.7388903, -74.0088418)",40.738890,-74.008842,POINT (-74.00884 40.73889)


In [29]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [30]:
gdf.to_file('output.geojson', driver='GeoJSON')

In [32]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Manhattan_Brooklyn_addresss_with_15+_Hamptons owners
